# 新闻视频处理演示

在这个笔记本中，我们将要针对一则来自江西湖口的本地新闻进行如下的处理：

1. 按照 `cut-scene` 来切割出若干镜头

1. 对上一步切割出来的镜头进行分类，识别出:

  - 演播室（没有字幕）
  - 演播室（带有字幕）
  - 其它

1. 识别带有字幕的演播室镜头中的字幕文本

1. 识别演播室镜头中，播音员播报的语音内容，将它转为文本

## 环境准备

我们已经准备好了一个 Conda 环境 `jxcn-newsvideo`，注意选择这个笔记本的 Kernel 为上诉环境！

- 如果是第一次运行，执行下面的 cell ，保证依赖包被正确安装到该笔记本是使用 Kernel
- 如果已经运行过一次，不必重复执行

In [3]:
%conda env update --file ../environment.yml

Solving environment: / 

Updating matplotlib is constricted by 

holoviz -> requires matplotlib==3.0.3

If you are sure you want an update of your package either try `conda update --all` or install a specific version of the package you want using `conda install <pkg>=<version>`



Updating pandas is constricted by 

holoviz -> requires pandas[version='<=0.24.2']

If you are sure you want an update of your package either try `conda update --all` or install a specific version of the package you want using `conda install <pkg>=<version>`

done
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Ran pip subprocess with arguments:
['/home/liuxy/miniconda3/envs/jxcn-newsvideo/bin/python', '-m', 'pip', 'install', '-U', '-r', '/home/liuxy/workspace/jxcn-newvideo-poc/condaenv.idqbzc04.requirements.txt']
Pip subprocess output:
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Requirement already up-to-date: scenedetect>=0.5 in /home/liuxy/miniconda3/envs/

## 上传要处理的视频文件

上传一个湖口的新闻视频，并指定其路径。

在下面的步骤中，我们将针对这个视频文件进行处理。

## 场景切割

## 场景分类

## 字幕转文本

## 语音转本文